# Self-Attention 与 KV Cache

## 1. Self-Attention 机制简介

Self-Attention（自注意力）是 Transformer 架构的核心。它允许模型在处理序列时，动态关注序列中不同位置的信息，实现信息的全局交互。

### 1.1 原理

- 对于输入序列中的每个 token，Self-Attention 会计算它与序列中所有 token 的相关性（注意力分数）。
- 通过加权求和的方式，融合其他 token 的信息，生成新的表示。

### 1.2 计算流程

1. 输入序列经过线性变换，得到 Query（Q）、Key（K）、Value（V）矩阵。
2. 计算 Q 与 K 的点积，得到注意力分数（score）。
3. 对分数进行 softmax，得到注意力权重。
4. 用权重加权 V，得到输出。

公式如下：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

### 1.3 多头注意力

- 多头注意力（Multi-Head Attention）将 Q、K、V 分成多组，分别计算注意力，最后拼接结果。
- 优点：能捕捉不同子空间的特征。

---

## 2. KV Cache（键值缓存）

### 2.1 背景

- 在推理阶段（如生成文本时），Transformer 需要逐步生成下一个 token。
- 每次生成时，理论上都要重新计算之前所有 token 的 Self-Attention，效率低下。

### 2.2 KV Cache 的作用

- **KV Cache** 是一种缓存机制，将之前计算得到的 Key（K）和 Value（V）保存下来。
- 新 token 只需计算自己的 Q，并与缓存的 K、V 进行注意力计算，无需重复计算历史部分。

### 2.3 优点

- 大幅提升推理速度，降低显存和计算消耗。
- 是大模型推理（如 ChatGPT、LLM 推理加速）的关键技术。

### 2.4 过程
在自回归解码（生成文本）过程中，我们的目标是 避免重复计算 K 和 V。KV Cache 采用增量缓存（Incremental Cache）的方式存储 Key 和 Value：

第一步（初始计算）：
计算第一个 token 的 K_1, V_1，存入缓存。
第二步（生成新 token 时）：
计算新 token 的 Query Q_n，但 不计算之前 token 的 Key 和 Value，直接从缓存读取 K_1, K_2, ..., K_{n-1} 和 V_1, V_2, ..., V_{n-1}。
只计算 Q_nK^T，然后进行注意力计算。
最终，存储 K 和 V，仅计算 Q，可以大幅减少计算量：

Attention ( Q n , [ K 1 , K 2 , . . . , K n − 1 ] , [ V 1 , V 2 , . . . , V n − 1 ] ) \text{Attention}(Q_n, [K_1, K_2, ..., K_{n-1}], [V_1, V_2, ..., V_{n-1}])


这样，计算复杂度由 O(N²) 降至 O(N)，随着序列长度增长，计算成本大幅降低。

---




**Multi-Head Attention 是顺序/位置不敏感的。**

$Q$ 中的每一个元素会和 $K$ 中所有个元素相乘并计算 Attention Score，这个的计算结果和 $K$ 中元素的顺序/位置是没有关系的，只和元素值的大小有关，因此 Multi-Head Attention 是对顺序/位置不敏感的——无论 $Q$ 和 $K$ 中元素的排列顺序如何其对应元素计算的结构都是恒定的、其计算的结果也无法反映其顺序/位置关系。

**Positional Encoding 的作用是解决 Multi-Head Attention 的顺序/位置不敏感性。**

通过给不同位置的元素加上一个能表示其顺序/位置值，将位置特征反应到了元素的特征值中，使得最终的计算结果是和元素的顺序/位置相关的——让模型利用到数据顺序/位置上的特征。

In [ ]:
import torch
import torch.nn.functional as F

def self_attention(Q, K, V, mask=None):
    d_k = Q.size(-1)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / d_k ** 0.5
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = F.softmax(scores, dim=-1)
    output = torch.matmul(attn, V)
    return output, attn

# KV Cache 示例
# 假设历史 Key/Value 已缓存
past_K = ...  # [batch, seq_len_past, dim]
past_V = ...  # [batch, seq_len_past, dim]
new_K = ...   # [batch, 1, dim]
new_V = ...   # [batch, 1, dim]

# 拼接历史和当前
K_cat = torch.cat([past_K, new_K], dim=1)
V_cat = torch.cat([past_V, new_V], dim=1)